# 1.0 Data Engineering with Python

## 1.1 Extracting and Transforming data

In this first section of the code we extract the data from INEGI. 

All urls are stored in the csv within disesases folder. 



In [ ]:

!pip config --user set global.progress_bar off
!pip install wget
!pip install simpledbf
!pip install dbfread
!pip install dbf

Writing to /root/.config/pip/pip.conf
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=35eeb14c24329abaf2e954a41cca98b53e979b32aa7637318dbc7ea8dbbc6ba2
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13800 sha256=b7dc79a574ec2242c6432282fc8c5dc7d39b9e7d2806d401936deb4f90c9f57c
  Stored in directory: /root/.cache/pip/wheels/24/43/f4/39ad84349e5358346be977fe626160f5625fdd3ea8e017518c
Successfully built simpledbf
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Load libraries
import pandas as pd;
import numpy as np;
import os;
import wget
import zipfile
import shutil
import glob
from simpledbf import Dbf5
import dbfread as dbr;
import dbf
import re

In [ ]:
######################################### Set up the environment #############################################
directory="./Resources"
os.chdir(directory)


Mounted at /content/gdrive


In [ ]:
####################################This function is done to transform dbf to csv ######################################3333
def dbf_to_csv(path):
  file_list=os.listdir()
  for file in file_list:
    if file.lower().endswith('.dbf'):
        dbf2 = Dbf5(file, codec='cp850')
        df2 = dbf2.to_dataframe()
        if (os.path.exists(path)==True):
          df2.to_csv(path+file[0:len(file)-4].lower()+".csv")
        else:
          os.mkdir(path)
          df2.to_csv(path+(file[0:len(file)-4].lower())+".csv")



In [ ]:
####################################This function is done to download files from INEGI ######################################3333
def download_files():
  directory="./Resources/raw"
  directory_clean="./Resources/clean"
  os.chdir(directory)
  urls_file = pd.read_csv('./Resources/raw/links.csv',header=None)
  urls_iterable=[]
  for row in urls_file.iterrows():
    if((row[1][1])-row[1][2]==0):
      temp_directory1=(directory+"/"+str(row[1][1]))
      os.mkdir(directory+"/"+str(row[1][1]))
      os.chdir(directory+"/"+str(row[1][1]))
      wget.download(row[1][0])
      zip_file=os.listdir()
      shutil.unpack_archive(zip_file[0])
      dbf_to_csv(directory_clean+"/"+str(row[1][1])+"/")
      subfolders_check=os.listdir()
      for subfold in subfolders_check:
        if not subfold.__contains__("."):
          temp_directory2=(temp_directory1+"/"+subfold)
          os.chdir(temp_directory2)
          dbf_to_csv(directory_clean+"/"+str(row[1][1])+"/")
    if((row[1][1])-row[1][2]<0):
      temp_directory1=(directory+"/"+str(row[1][1]))
      os.mkdir(directory+"/"+str(row[1][1]))
      os.chdir(directory+"/"+str(row[1][1]))
      wget.download(row[1][0])
      zip_file=os.listdir()
      shutil.unpack_archive(zip_file[0])
      dbf_to_csv(directory_clean+"/"+str(row[1][1]))
      subfolders_check=os.listdir()
      for subfold in subfolders_check:
        if not subfold.__contains__("."):
          temp_directory2=(temp_directory1+"/"+subfold)
          year_subfold=re.findall(r'\d+',subfold)
          os.chdir(temp_directory2)
          dbf_to_csv(directory_clean+"/"+year_subfold[0]+"/")



In [ ]:

download_files()

Mounted at /content/gdrive


## 1.2 Transforming data

In this section we append all the csv files we previously downloaded and transformed into eighth different tables. 

In [ ]:
tables=["gpolimex", 
        "parentesco", 
        "capgpo",
        "listamex",
        "lista1",
        "catminde"]

time_var=["defun", "catemlde"]


In [ ]:
directory_clean="./Resources/clean"
os.chdir(directory_clean)
years=["2020","2019","2018","2017","2016","2015","2014","2013"]
for year in years:
  os.chdir(directory_clean+"/"+year)
  print(os.listdir())
  for table in time_var:
    temp_table=pd.read_csv(table+year[2:4]+".csv")
    temp_table["year"]=year
    temp_table.append(temp_table)
  temp_table.to_csv(directory_clean+"/"+table+".csv")

['lista1.csv', 'listamex.csv', 'parentesco.csv', 'capgpo.csv', 'catemlde20.csv', 'catminde.csv', 'defun20.csv', 'gpolimex.csv']
['capgpo.csv', 'catemlde19.csv', 'catminde.csv', 'defun19.csv', 'gpolimex.csv', 'lista1.csv', 'listamex.csv', 'parentesco.csv']
['defun18.csv', 'gpolimex.csv', 'lista1.csv', 'listamex.csv', 'parentesco.csv', 'capgpo.csv', 'catemlde18.csv', 'catminde.csv']
['listamex.csv', 'parentesco.csv', 'capgpo.csv', 'catemlde17.csv', 'catminde.csv', 'defun17.csv', 'gpolimex.csv', 'lista1.csv']
['gpolimex.csv', 'lista1.csv', 'listamex.csv', 'parentesco.csv', 'defun16.csv', 'capgpo.csv', 'catemlde16.csv', 'catminde.csv']
['lista1.csv', 'listamex.csv', 'defun15.csv', 'parentesco.csv', 'catemlde15.csv', 'catminde.csv', 'capgpo.csv', 'gpolimex.csv']
['capgpo.csv', 'catemlde14.csv', 'catminde.csv', 'defun14.csv', 'gpolimex.csv', 'lista1.csv', 'listamex.csv', 'parentesco.csv']
['capgpo.csv', 'catemlde13.csv', 'catminde.csv', 'defun13.csv', 'gpolimex.csv', 'lista1.csv', 'listamex.

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (43,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
directory_clean="/content/gdrive/My Drive/diseases/clean"
os.chdir(directory_clean)
years=["2020","2019","2018","2017","2016","2015","2014","2013"]
#years=os.listdir()
for year in years:
  os.chdir(directory_clean+"/"+year)
  print(os.listdir())
  for table in tables:
    temp_table=pd.read_csv(table+".csv")
    temp_table["year"]=year
    temp_table.append(temp_table)
    temp_table.to_csv(directory_clean+"/"+table+".csv")

['lista1.csv', 'listamex.csv', 'parentesco.csv', 'capgpo.csv', 'catemlde20.csv', 'catminde.csv', 'defun20.csv', 'gpolimex.csv']
['capgpo.csv', 'catemlde19.csv', 'catminde.csv', 'defun19.csv', 'gpolimex.csv', 'lista1.csv', 'listamex.csv', 'parentesco.csv']
['defun18.csv', 'gpolimex.csv', 'lista1.csv', 'listamex.csv', 'parentesco.csv', 'capgpo.csv', 'catemlde18.csv', 'catminde.csv']
['listamex.csv', 'parentesco.csv', 'capgpo.csv', 'catemlde17.csv', 'catminde.csv', 'defun17.csv', 'gpolimex.csv', 'lista1.csv']
['gpolimex.csv', 'lista1.csv', 'listamex.csv', 'parentesco.csv', 'defun16.csv', 'capgpo.csv', 'catemlde16.csv', 'catminde.csv']
['lista1.csv', 'listamex.csv', 'defun15.csv', 'parentesco.csv', 'catemlde15.csv', 'catminde.csv', 'capgpo.csv', 'gpolimex.csv']
['capgpo.csv', 'catemlde14.csv', 'catminde.csv', 'defun14.csv', 'gpolimex.csv', 'lista1.csv', 'listamex.csv', 'parentesco.csv']
['capgpo.csv', 'catemlde13.csv', 'catminde.csv', 'defun13.csv', 'gpolimex.csv', 'lista1.csv', 'listamex.

In [ ]:
temp_lista1=pd.read_csv('/content/gdrive/My Drive/diseases/clean/2020/lista1.csv')
temp_lista1["year"]="2020"
temp_listamex=pd.read_csv('/content/gdrive/My Drive/diseases/clean/2020/listamex.csv')
temp_listamex["year"]="2020"
temp_parentesco=pd.read_csv('/content/gdrive/My Drive/diseases/clean/2020/parentesco.csv')
temp_parentesco["year"]="2020"
temp_capgpo=pd.read_csv('/content/gdrive/My Drive/diseases/clean/2020/capgpo.csv')
temp_capgpo["year"]="2020"
temp_catemlde=pd.read_csv('/content/gdrive/My Drive/diseases/clean/2020/catemlde20.csv')
temp_catemlde["year"]="2020"
temp_catminde=pd.read_csv('/content/gdrive/My Drive/diseases/clean/2020/catminde.csv')
temp_catminde["year"]="2020"
temp_defun=pd.read_csv('/content/gdrive/My Drive/diseases/clean/2020/defun20.csv')
temp_defun["year"]="2020"
temp_gpolimex=pd.read_csv('/content/gdrive/My Drive/diseases/clean/2020/gpolimex.csv')
temp_gpolimex["year"]="2020"



In [ ]:
#####Lista1
directory_clean="./Resources/clean"
os.chdir(directory_clean)
years=["2019","2018","2017","2016","2015"]
#years=os.listdir()
tables=["lista1"]
for table in tables:
  for year in years:
    os.chdir(directory_clean+"/"+year)
    temp_table=pd.read_csv(table+".csv")
    temp_table["year"]=year
    temp_lista1=temp_lista1.append(temp_table)
  temp_lista1.to_csv(directory_clean+"/"+table+".csv", index = False)

In [ ]:
#####temp_listamex
directory_clean="./Resources/clean"
os.chdir(directory_clean)
years=["2019","2018","2017","2016","2015"]
#years=os.listdir()
tables=["listamex"]
for table in tables:
  for year in years:
    os.chdir(directory_clean+"/"+year)
    temp_table=pd.read_csv(table+".csv")
    temp_table["year"]=year
    temp_listamex=temp_listamex.append(temp_table)
temp_listamex.to_csv(directory_clean+"/"+table+".csv")

In [ ]:
#####temp_parentesco
directory_clean="./Resources/clean"
os.chdir(directory_clean)
years=["2019","2018","2017","2016","2015"]
#years=os.listdir()
tables=["parentesco"]
for table in tables:
  for year in years:
    os.chdir(directory_clean+"/"+year)
    temp_table=pd.read_csv(table+".csv")
    temp_table["year"]=year
    temp_parentesco=temp_parentesco.append(temp_table)
temp_parentesco.to_csv(directory_clean+"/"+table+".csv")

In [ ]:
#####temp_capgpo
directory_clean="/content/gdrive/My Drive/diseases/clean"
os.chdir(directory_clean)
years=["2019","2018","2017","2016","2015"]
#years=os.listdir()
tables=["capgpo"]
for table in tables:
  for year in years:
    os.chdir(directory_clean+"/"+year)
    temp_table=pd.read_csv(table+".csv")
    temp_table["year"]=year
    temp_capgpo=temp_capgpo.append(temp_table)
temp_capgpo.to_csv(directory_clean+"/"+table+".csv")

In [ ]:
#####temp_catminde
directory_clean="./Resources/clean"
os.chdir(directory_clean)
years=["2019","2018","2017","2016","2015"]
#years=os.listdir()
tables=["catminde"]
for table in tables:
  for year in years:
    os.chdir(directory_clean+"/"+year)
    temp_table=pd.read_csv(table+".csv")
    temp_table["year"]=year
    temp_catminde=temp_catminde.append(temp_table)
temp_catminde.to_csv(directory_clean+"/"+table+".csv")

In [ ]:
#####temp_gpolimex
directory_clean="./Resources/clean"
os.chdir(directory_clean)
years=["2019","2018","2017","2016","2015"]
#years=os.listdir()
tables=["gpolimex"]
for table in tables:
  for year in years:
    os.chdir(directory_clean+"/"+year)
    temp_table=pd.read_csv(table+".csv")
    temp_table["year"]=year
    temp_gpolimex=temp_gpolimex.append(temp_table)
temp_gpolimex.to_csv(directory_clean+"/"+table+".csv")

In [ ]:
#####temp_catemlde
directory_clean="./Resources/clean"
os.chdir(directory_clean)
years=["2019","2018","2017","2016","2015"]
#years=os.listdir()
tables=["catemlde"]
for table in tables:
  for year in years:
    os.chdir(directory_clean+"/"+year)
    temp_table=pd.read_csv(table+year[2:4]+".csv")
    temp_table["year"]=year
    temp_catemlde=temp_catemlde.append(temp_table)
temp_catemlde.to_csv(directory_clean+"/"+table+".csv")

In [ ]:
#####temp_defun
directory_clean="./Resources/clean"
os.chdir(directory_clean)
years=["2019","2018","2017","2016","2015"]
#years=os.listdir()
tables=["defun"]
for table in tables:
  for year in years:
    os.chdir(directory_clean+"/"+year)
    temp_table=pd.read_csv(table+year[2:4]+".csv")
    temp_table["year"]=year
    temp_defun=temp_defun.append(temp_table)
  temp_defun.to_csv(directory_clean+"/"+table+".csv")

2019
2018
2017
2016
2015
